# Project Title: Feature Extraction And Opinion Mining for Summarization

## Guidelines: 
### Use modular style of programming i.e. write the program as a series of function
## DON'T Use Camel Case, Use UNDERSCORE
### variable names should be indicate the data stored in it
### Add comments wherever required to improve readability of code

## Web scraping:
### input = product_id 
### output: List of reviews

## Enhancements: Calculate number of pages of reviews so that the number of inputed reviews can be increased (Assigned to Gaurav Ghongde) 


In [96]:
from lxml import html  
import requests
import pandas as pd
import time
product_id = 'B01M0GB8CC'  ## Fetched from user inputted url {Assigned to: Hrushikesh Gate }


amazon_url = 'https://www.amazon.com/product-reviews/'+ product_id + '?pageNumber=' + str(j)
user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
headers = {'User-Agent': user_agent}
page = requests.get(amazon_url, headers = headers)
parser = html.fromstring(page.content)

#xpath_pagination_bar = '//div[@data-hook="pagination-bar"]'
#pagination_bar = parser.xpath(xpath_pagination_bar)

page_list = []

for el in parser.xpath('//div[@data-hook="pagination-bar"]/ul/li[@data-reftag="cm_cr_arp_d_paging_btm"]//text()'):
    page_list.append(int(el))
    
page_count = max(page_list)
if page_count>10:
    page_count = 10

for j in range(1,page_count):
    try:
        time.sleep(2)
        amazon_url = 'https://www.amazon.com/product-reviews/'+ product_id + '?pageNumber=' + str(j)
        print(amazon_url)
        user_agent = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'
        if(j==1):
            reviews_df = pd.DataFrame()

        headers = {'User-Agent': user_agent}
        page = requests.get(amazon_url, headers = headers)
        parser = html.fromstring(page.content)

        xpath_reviews = '//div[@data-hook="review"]'
        reviews = parser.xpath(xpath_reviews)

        xpath_rating  = './/i[@data-hook="review-star-rating"]//text()' 
        xpath_title   = './/a[@data-hook="review-title"]//text()'
        xpath_author  = './/a[@data-hook="review-author"]//text()'
        xpath_date    = './/span[@data-hook="review-date"]//text()'
        xpath_body    = './/span[@data-hook="review-body"]//text()'
        xpath_helpful = './/span[@data-hook="helpful-vote-statement"]//text()'

        for review in reviews:
            rating  = review.xpath(xpath_rating)
            title   = review.xpath(xpath_title)
            author  = review.xpath(xpath_author)
            date    = review.xpath(xpath_date)
            body    = review.xpath(xpath_body)
            helpful = review.xpath(xpath_helpful)

            review_dict = {'rating': rating,
                           'title': title,
                           'author': author,             
                           'date': date,
                           'body': body,
                           'helpful': helpful}
            reviews_df = reviews_df.append(review_dict, ignore_index=True)
    except requests.exceptions.ConnectionError:
        print("Connection refused")
reviews_df.head()

https://www.amazon.com/product-reviews/B01M0GB8CC?pageNumber=1
https://www.amazon.com/product-reviews/B01M0GB8CC?pageNumber=2
https://www.amazon.com/product-reviews/B01M0GB8CC?pageNumber=3
https://www.amazon.com/product-reviews/B01M0GB8CC?pageNumber=4
https://www.amazon.com/product-reviews/B01M0GB8CC?pageNumber=5
https://www.amazon.com/product-reviews/B01M0GB8CC?pageNumber=6
https://www.amazon.com/product-reviews/B01M0GB8CC?pageNumber=7
https://www.amazon.com/product-reviews/B01M0GB8CC?pageNumber=8
https://www.amazon.com/product-reviews/B01M0GB8CC?pageNumber=9


author                                               body  \
0     []  [After losing my iPhone7 EarPods, I bought the...   
1     []  [I use my earbuds for singing and immediately ...   
2     []  [I just got the headphones like this ! Open an...   
3     []  [Not original apple earpods. Dont waste yr mon...   
4     []  [I bought 3 pairs of these headphones for Chri...   

                  date                          helpful                rating  \
0   [January 29, 2017]  [551 people found this helpful]  [4.0 out of 5 stars]   
1  [December 28, 2017]   [98 people found this helpful]  [1.0 out of 5 stars]   
2   [November 4, 2017]  [153 people found this helpful]  [1.0 out of 5 stars]   
3  [December 19, 2017]   [53 people found this helpful]  [1.0 out of 5 stars]   
4     [March 18, 2018]   [38 people found this helpful]  [1.0 out of 5 stars]   

                                               title  
0  [Important: Prime-Exclusive Amazon listing is ...  
1                     [WARNING: THESE ARE NOT APPLE]  
2                                 [Is this a joke??]  
3                                    [Not originals]  
4  [I bought 3 pairs of these headphones for Chri...

### Number of reviews fetched

In [97]:
count= reviews_df.count()
review_cnt = count[1]
review_cnt


90

## Converting reviews into HTML format for ibm watson discovery

In [98]:
input_string= ""
input_text=""
for i in range(review_cnt):
    input_string= input_string + "<h>" + reviews_df['title'][i][0] + "</h>"
    input_string= input_string + " <p> " + reviews_df['body'][i][0] + " </p> " 
    input_text= input_text + reviews_df['title'][i][0] + " " + reviews_df['body'][i][0]  + " 8888888 "


In [99]:
reviews_df['text'] = reviews_df['title'] + reviews_df['body']

## Keyword extraction using IBM watson 

In [41]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, KeywordsOptions
try:
    natural_language_understanding = NaturalLanguageUnderstandingV1(
        version='2018-11-16',
        iam_apikey='gm6iXTWNo8uw12AcIx4LVjkNt39APWHgFS1py2w4Zzki',
        url='https://gateway-wdc.watsonplatform.net/natural-language-understanding/api'
    )

    response = natural_language_understanding.analyze(
        html="<html><p>"+ input_string + "<p></html>",
        features=Features(keywords=KeywordsOptions(sentiment=True,emotion=True,limit=5))).get_result()

    print(json.dumps(response, indent=2)) ##to be written on a json file
except requests.exceptions.ConnectionError:
    print("Connection refused")

{
  "usage": {
    "text_units": 3,
    "text_characters": 21207,
    "features": 1
  },
  "language": "en",
  "keywords": [
    {
      "text": "legit Apple products",
      "sentiment": {
        "score": -0.926724,
        "label": "negative"
      },
      "relevance": 0.664305,
      "emotion": {
        "sadness": 0.450279,
        "joy": 0.026799,
        "fear": 0.076955,
        "disgust": 0.058949,
        "anger": 0.04515
      },
      "count": 1
    },
    {
      "text": "couple months use",
      "sentiment": {
        "score": -0.922524,
        "label": "negative"
      },
      "relevance": 0.58577,
      "emotion": {
        "sadness": 0.065787,
        "joy": 0.70563,
        "fear": 0.08373,
        "disgust": 0.021548,
        "anger": 0.161277
      },
      "count": 1
    },
    {
      "text": "Great buy",
      "sentiment": {
        "score": 0.999003,
        "label": "positive"
      },
      "relevance": 0.581801,
      "emotion": {
        "sadness": 0.035

## Text Cleaning

In [100]:
import nltk
import re
input_text = re.sub(r'[^\w\s]','',input_text)
from nltk.tokenize import sent_tokenize, word_tokenize
sentence = sent_tokenize(input_text)
review_subtext = input_text.split("8888888")

In [101]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
tfidf_input=""
for review in review_subtext:
    temp_review = review.split(" ")
    filtered_words = [w for w in temp_review if not w in stop_words]
    filtered_words = list(filter(None, filtered_words))
    temp= nltk.pos_tag(filtered_words)
    for k in temp:
        if(k[1]=='NN' or k[1]=='NNS'):
            tfidf_input = tfidf_input + k[0] + " "
    tfidf_input = tfidf_input + "|" + " "

##  Generate input for tfidf

In [102]:
tfidf_input = tfidf_input.split("|")
tfidf_input.remove(' ')
tfidf_input = list(filter(None,tfidf_input))
#print(tfidf_input)

In [103]:
from sklearn.feature_extraction.text import TfidfVectorizer 
corpus = tfidf_input
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', min_df = 0, stop_words = 'english')

tfidf_matrix =  tf.fit_transform(corpus)
feature_names = tf.get_feature_names() 
feature_names

['afraid',
 'amazon',
 'apple',
 'arrive',
 'attach',
 'audio',
 'background',
 'bag',
 'base',
 'basing',
 'bass',
 'benefit',
 'bit',
 'box',
 'brand',
 'break',
 'bucks',
 'buds',
 'buggy',
 'buttons',
 'buy',
 'buying',
 'cable',
 'cancellation',
 'cardboard',
 'carpet',
 'case',
 'cause',
 'chance',
 'change',
 'cheap',
 'chords',
 'cloths',
 'comparison',
 'complaints',
 'condition',
 'connection',
 'connector',
 'container',
 'control',
 'controls',
 'cord',
 'cords',
 'corrodedphoto',
 'cost',
 'counterfeit',
 'couple',
 'crap',
 'customer',
 'cutting',
 'daughter',
 'daughters',
 'day',
 'days',
 'deal',
 'dealbreaker',
 'decent',
 'design',
 'desk',
 'didnt',
 'difference',
 'differences',
 'disappointment',
 'documentation',
 'doesnt',
 'dollars',
 'dont',
 'dork',
 'doubt',
 'ear',
 'earbud',
 'earbuds',
 'earphones',
 'earpods',
 'ears',
 'electronics',
 'end',
 'excuses',
 'fake',
 'fakes',
 'family',
 'feel',
 'feet',
 'filth',
 'fingers',
 'galaxy',
 'garbage',
 'garbag

## Using simple term frequency

In [107]:
word_token= word_tokenize(input_text)
import re
input_text2 = re.sub(r'[^\w\s]','',input_text)

In [108]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
#import collections
#print(stop_words)
filtered_words = [w for w in word_token if not w in stop_words and w != "8888888"]


In [109]:
temp= nltk.pos_tag(filtered_words)


In [110]:
from scipy.stats import itemfreq
high_freq_words = []
noun_words= []
for i in temp:
    if(i[1]== 'NN' or i[1]=='NNS'):
        noun_words.append(i[0])
noun_freq = itemfreq(noun_words)
for j in noun_freq:
    if(int(j[1])> 2):
        high_freq_words.append(j)
high_freq_words

C:\Users\Hareesh\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: `itemfreq` is deprecated!
`itemfreq` is deprecated and will be removed in a future version. Use instead `np.unique(..., return_counts=True)`
  import sys


[array(['anything', '4'], dtype='<U21'),
 array(['apple', '21'], dtype='<U21'),
 array(['bag', '3'], dtype='<U21'),
 array(['box', '7'], dtype='<U21'),
 array(['brand', '4'], dtype='<U21'),
 array(['buds', '3'], dtype='<U21'),
 array(['buy', '3'], dtype='<U21'),
 array(['buying', '3'], dtype='<U21'),
 array(['case', '3'], dtype='<U21'),
 array(['cheap', '3'], dtype='<U21'),
 array(['connection', '3'], dtype='<U21'),
 array(['connector', '5'], dtype='<U21'),
 array(['control', '3'], dtype='<U21'),
 array(['daughter', '3'], dtype='<U21'),
 array(['day', '4'], dtype='<U21'),
 array(['days', '4'], dtype='<U21'),
 array(['didnt', '3'], dtype='<U21'),
 array(['difference', '4'], dtype='<U21'),
 array(['disappointment', '3'], dtype='<U21'),
 array(['dont', '3'], dtype='<U21'),
 array(['ear', '7'], dtype='<U21'),
 array(['earbud', '4'], dtype='<U21'),
 array(['earbuds', '8'], dtype='<U21'),
 array(['earphones', '7'], dtype='<U21'),
 array(['ears', '3'], dtype='<U21'),
 array(['headphones', '42

## Bigrams


[]


## Find Sentiment

In [31]:
import json
from watson_developer_cloud import NaturalLanguageUnderstandingV1
from watson_developer_cloud.natural_language_understanding_v1 import Features, SentimentOptions
noun_features= ['ipad', 'iphone'] ## to be replaced by features extracted
def find_sentiment(noun_features):
    try:
        natural_language_understanding = NaturalLanguageUnderstandingV1(
            version='2018-11-16',
            iam_apikey='gm6iXTWNo8uw12AcIx4LVjkNt39APWHgFS1py2w4Zzki',
            url='https://gateway-wdc.watsonplatform.net/natural-language-understanding/api'
        )

        response = natural_language_understanding.analyze(
            html="<html>"+ input_string + "</html>",
            features=Features(sentiment=SentimentOptions(targets=noun_features))).get_result()

        print(json.dumps(response, indent=2)) ##to be written on a json file
    except requests.exceptions.ConnectionError:
        print("Connection refused")
        
        
noun_features= ['apple', 'quality'] ## to be replaced by features extracted
find_sentiment(noun_features)
        

{
  "usage": {
    "text_units": 2,
    "text_characters": 10951,
    "features": 1
  },
  "sentiment": {
    "targets": [
      {
        "text": "apple",
        "score": -0.480431,
        "mixed": "1",
        "label": "negative"
      },
      {
        "text": "quality",
        "score": -0.479675,
        "mixed": "1",
        "label": "negative"
      }
    ],
    "document": {
      "score": -0.509542,
      "label": "negative"
    }
  },
  "language": "en"
}
